In [1]:
import sys
sys.path.append('../')
from classes.classifiers import MainSpeakerIntensity5Classifier
from classes.datasets import FeatureDataset

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import numpy as np
import random
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

In [2]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [3]:
def train_model(data_dir, features_names, model_name, epochs=20, batch_size=32, learning_rate=0.001, data_split_coeff=0.7):
    writer = SummaryWriter(comment=model_name)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Data Loaders
    dataset = FeatureDataset(data_dir, data_dir, features_names)
    train_dataset, val_dataset = random_split(dataset, [round(len(dataset) * data_split_coeff), len(dataset) - round(len(dataset) * data_split_coeff)])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Model, Loss, Optimizer
    model = MainSpeakerIntensity5Classifier(number_of_features=len(features_names)).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    best_accuracy = 0
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{epochs}"):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")
        writer.add_scalar("Loss/train", running_loss, epoch)

        # Validace
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc="Validating"):
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f"Validation Accuracy: {accuracy}")
        writer.add_scalar("Accuracy/train", accuracy, epoch)

        # Ulozeni modelu
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), f"../models/{model_name}.pt")
            print(f"Model saved with accuracy: {accuracy}")
        print("--------------------------------------------------")

    writer.flush()
    writer.close()


In [4]:
data_directory = "../preprocessed_data/"
train_model(data_directory, ["ipd17_"], "ipd17_model", epochs=12, batch_size=32, learning_rate=0.001, data_split_coeff=0.7)

Using device: cuda


Training Epoch 1/12: 100%|██████████| 27388/27388 [30:26<00:00, 14.99it/s]


Epoch 1, Loss: 0.9018936695165664


Validating: 100%|██████████| 11738/11738 [08:55<00:00, 21.91it/s]


Validation Accuracy: 0.7205324813631523
Model saved with accuracy: 0.7205324813631523
--------------------------------------------------


...

Validation Accuracy: 0.748035143769968
Model saved with accuracy: 0.748035143769968
--------------------------------------------------


Training Epoch 12/12: 100%|██████████| 27388/27388 [29:15<00:00, 15.60it/s]


Epoch 12, Loss: 0.5407873904967727


Validating: 100%|██████████| 11738/11738 [08:58<00:00, 21.79it/s]

Validation Accuracy: 0.7460330138445155
--------------------------------------------------


In [ ]:
data_directory = "../preprocessed_data/"
train_model(data_directory, ["ipd35_"], "ipd35_model", epochs=12, batch_size=32, learning_rate=0.001, data_split_coeff=0.7)

Using device: cuda


Training Epoch 1/12: 100%|██████████| 27388/27388 [30:09<00:00, 15.13it/s]


Epoch 1, Loss: 0.9441120435149283


Validating: 100%|██████████| 11738/11738 [09:22<00:00, 20.88it/s]


Validation Accuracy: 0.6981469648562301
Model saved with accuracy: 0.6981469648562301
--------------------------------------------------


...

Validation Accuracy: 0.7272949946751863
Model saved with accuracy: 0.7272949946751863
--------------------------------------------------


Training Epoch 12/12: 100%|██████████| 27388/27388 [30:20<00:00, 15.05it/s]


Epoch 12, Loss: 0.6648986146563471


Validating: 100%|██████████| 11738/11738 [09:24<00:00, 20.78it/s]

Validation Accuracy: 0.7229925452609158
--------------------------------------------------


In [ ]:
data_directory = "../preprocessed_data/"
train_model(data_directory, ["ipd14_", "ipd17_"], "ipd14_ipd17_model", epochs=12, batch_size=32, learning_rate=0.001, data_split_coeff=0.7)

Using device: cuda


Training Epoch 1/12: 100%|██████████| 27388/27388 [1:27:44<00:00,  5.20it/s]  


Epoch 1, Loss: 0.8804607911657569


Validating: 100%|██████████| 11738/11738 [17:26<00:00, 11.22it/s]


Validation Accuracy: 0.7303168264110756
Model saved with accuracy: 0.7303168264110756
--------------------------------------------------


...

Validation Accuracy: 0.7582082002129925
--------------------------------------------------


Training Epoch 12/12: 100%|██████████| 27388/27388 [1:11:22<00:00,  6.40it/s]


Epoch 12, Loss: 0.537393873504945


Validating: 100%|██████████| 11738/11738 [16:01<00:00, 12.20it/s]

Validation Accuracy: 0.7595527156549521
--------------------------------------------------


In [ ]:
data_directory = "../preprocessed_data/"
train_model(data_directory, ["ipd17_", "ild17_"], "ipd17_ild17_model_ii", epochs=12, batch_size=32, learning_rate=0.001, data_split_coeff=0.7)

Using device: cuda


Training Epoch 1/12: 100%|██████████| 27388/27388 [1:03:35<00:00,  7.18it/s]


Epoch 1, Loss: 0.8806308119992923


Validating: 100%|██████████| 11738/11738 [14:31<00:00, 13.47it/s]


Validation Accuracy: 0.7212752928647498
Model saved with accuracy: 0.7212752928647498
--------------------------------------------------


...

Validation Accuracy: 0.7519062832800852
--------------------------------------------------


Training Epoch 12/12: 100%|██████████| 27388/27388 [47:56<00:00,  9.52it/s] 


Epoch 12, Loss: 0.5982065691644082


Validating: 100%|██████████| 11738/11738 [14:39<00:00, 13.35it/s]

Validation Accuracy: 0.7515202342917998
--------------------------------------------------


In [ ]:
data_directory = "../preprocessed_data/"
train_model(data_directory, ["ipd14_", "ipd17_", "ipd35_"], "ipd14_ipd17_ipd35_model", epochs=12, batch_size=32, learning_rate=0.001, data_split_coeff=0.7)

Using device: cuda


Training Epoch 1/12: 100%|██████████| 27388/27388 [1:45:14<00:00,  4.34it/s]


Epoch 1, Loss: 0.8747032030239512


Validating: 100%|██████████| 11738/11738 [36:45<00:00,  5.32it/s]


Validation Accuracy: 0.7252076677316294
Model saved with accuracy: 0.7252076677316294
--------------------------------------------------


...

Validation Accuracy: 0.7628780617678381
--------------------------------------------------


Training Epoch 12/12: 100%|██████████| 27388/27388 [1:27:04<00:00,  5.24it/s]


Epoch 12, Loss: 0.54674851725579


Validating: 100%|██████████| 11738/11738 [33:05<00:00,  5.91it/s]

Validation Accuracy: 0.7574014909478168
--------------------------------------------------


In [ ]:
data_directory = "../preprocessed_data/"
train_model(data_directory, ["ild17_"], "ild17_model", epochs=12, batch_size=32, learning_rate=0.001, data_split_coeff=0.7)

Using device: cuda


Training Epoch 1/12: 100%|██████████| 27388/27388 [29:04<00:00, 15.70it/s]


Epoch 1, Loss: 0.9353419837963673


Validating: 100%|██████████| 11738/11738 [09:48<00:00, 19.94it/s]


Validation Accuracy: 0.7014483493077742
Model saved with accuracy: 0.7014483493077742
--------------------------------------------------


...

Validation Accuracy: 0.724164004259851
--------------------------------------------------


Training Epoch 12/12: 100%|██████████| 27388/27388 [28:20<00:00, 16.10it/s]


Epoch 12, Loss: 0.6863019313269877


Validating: 100%|██████████| 11738/11738 [09:43<00:00, 20.11it/s]

Validation Accuracy: 0.7294435569755059
--------------------------------------------------


In [ ]:
data_directory = "../preprocessed_data/"
train_model(data_directory, ["ild14_", "ild17_"], "ild14_ild17_model", epochs=12, batch_size=32, learning_rate=0.001, data_split_coeff=0.7)

Using device: cuda


Training Epoch 1/12: 100%|██████████| 27388/27388 [1:10:23<00:00,  6.48it/s]


Epoch 1, Loss: 0.9095842073359628


Validating: 100%|██████████| 11738/11738 [17:45<00:00, 11.01it/s]


Validation Accuracy: 0.7215415335463259
Model saved with accuracy: 0.7215415335463259
--------------------------------------------------


...

Validation Accuracy: 0.7464376996805112
Model saved with accuracy: 0.7464376996805112
--------------------------------------------------


Training Epoch 12/12: 100%|██████████| 27388/27388 [52:09<00:00,  8.75it/s]  


Epoch 12, Loss: 0.6669935131883781


Validating: 100%|██████████| 11738/11738 [15:03<00:00, 12.99it/s]

Validation Accuracy: 0.746104898828541
--------------------------------------------------


In [4]:
data_directory = "../preprocessed_data/"
train_model(data_directory, ["ild35_"], "ild35_model", epochs=12, batch_size=32, learning_rate=0.001, data_split_coeff=0.7)

Using device: cuda


Training Epoch 1/12: 100%|██████████| 27388/27388 [34:08<00:00, 13.37it/s] 


Epoch 1, Loss: 0.931524224969981


Validating: 100%|██████████| 11738/11738 [09:10<00:00, 21.31it/s]


Validation Accuracy: 0.7028860489882854
Model saved with accuracy: 0.7028860489882854
--------------------------------------------------


...

Validation Accuracy: 0.718069755058573
--------------------------------------------------


Training Epoch 12/12: 100%|██████████| 27388/27388 [29:44<00:00, 15.35it/s]


Epoch 12, Loss: 0.7499192103376846


Validating: 100%|██████████| 11738/11738 [09:06<00:00, 21.46it/s]

Validation Accuracy: 0.717433439829606
--------------------------------------------------


In [5]:
data_directory = "../preprocessed_data/"
train_model(data_directory, ["ild14_", "ild17_", "ild35_"], "ild14_ild17_ild35_model", epochs=12, batch_size=32, learning_rate=0.001, data_split_coeff=0.7)

Using device: cuda


Training Epoch 1/12: 100%|██████████| 27388/27388 [1:34:46<00:00,  4.82it/s]


Epoch 1, Loss: 0.8861154270657218


Validating: 100%|██████████| 11738/11738 [36:56<00:00,  5.30it/s]


Validation Accuracy: 0.723069755058573
Model saved with accuracy: 0.723069755058573
--------------------------------------------------


...

Validation Accuracy: 0.7493184238551651
Model saved with accuracy: 0.7493184238551651
--------------------------------------------------


Training Epoch 12/12: 100%|██████████| 27388/27388 [1:34:07<00:00,  4.85it/s]


Epoch 12, Loss: 0.6787923428741284


Validating: 100%|██████████| 11738/11738 [33:09<00:00,  5.90it/s]

Validation Accuracy: 0.7474707135250266
--------------------------------------------------
